<a href="https://colab.research.google.com/github/purrchinelearning/basic-pytorch/blob/main/InceptionV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchsummary import summary
from torchvision import transforms,datasets
from torch.utils.data import TensorDataset, DataLoader, Dataset
import cv2
from PIL import Image
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
class conv(nn.Module):
  def __init__(self, in_channel, out_channel, kernel_size, **kwargs):
    super(conv,self).__init__()
    self.Conv = nn.Conv2d(in_channel,out_channel,kernel_size, **kwargs)
    self.Batch = nn.BatchNorm2d(out_channel)
    self.Act = nn.ReLU()
  def forward(self,x):
    x = self.Conv(x)
    x = self.Batch(x)
    x = self.Act(x)
    return x

class Incep1(nn.Module):
  def __init__(self, ch):
    super(Incep1,self).__init__()
    #ch = [p1_1in, p1_1out, p1_2out, p1_3out, p2_1in, p2_1out, p2_2out, p3_2in, p3_2out, p4in, p4out]
    self.path1_1 = conv(ch[0], ch[1], 1)
    self.path1_2 = conv(ch[1], ch[2], 3)
    self.path1_3 = conv(ch[2], ch[3], 3)
    self.path2_1 = conv(ch[4], ch[5], 1)
    self.path2_2 = conv(ch[5], ch[6], 3)
    self.path3_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    self.path3_2 = conv(ch[7],ch[8],1)
    self.path4 = conv(ch[9],ch[10],1)
  def forward(self,x):
    x1 = self.path1_3(self.path1_2(self.path1_1(x)))
    x2 = self.path2_2(self.path2_1(x))
    x3 = self.path3_2(self.path3_1(x))
    x4 = self.path4(x)
    return torch.cat([x1,x2,x3,x4],1)

class Incep2(nn.Module):
  def __init__(self, ch):
    super(Incep2,self).__init__()
    #ch = [p1_1in, p1_1out, p1_2out, p1_3out, p1_4out, p1_5out,
    #      p2_1in, p2_1out, p2_2out, p2_3out, p3_2in, p3_2out, p4in, p4out]
    self.path1_1 = conv(ch[0], ch[1], 1)
    self.path1_2 = conv(ch[1], ch[2], (1,7))
    self.path1_3 = conv(ch[2], ch[3], (7,1))
    self.path1_4 = conv(ch[3], ch[4], (1,7))
    self.path1_5 = conv(ch[4], ch[5], (7,1))
    self.path2_1 = conv(ch[6], ch[7], 1)
    self.path2_2 = conv(ch[7], ch[8], (1,7))
    self.path2_3 = conv(ch[8], ch[9], (7,1))
    self.path3_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    self.path3_2 = conv(ch[10],ch[11],1)
    self.path4 = conv(ch[12],ch[13],1)
  def forward(self,x):
    x1 = self.path1_5(self.path1_4(self.path1_3(self.path1_2(self.path1_1(x)))))
    x2 = self.path2_3(self.path2_2(self.path2_1(x)))
    x3 = self.path3_2(self.path3_1(x))
    x4 = self.path4(x)
    return torch.cat([x1,x2,x3,x4],1)

class Incep3(nn.Module):
  def __init__(self, ch):
    super(Incep3,self).__init__()
    #ch = [p1_1in, p1_1out, p1_2out, p1_3_1out, p1_3_2out, p2_1in, p2_1out, p2_2_1out, p2_2_2out,
    #      p3_2in, p3_2out, p4in, p4out]
    self.path1_1 = conv(ch[0], ch[1], 1)
    self.path1_2 = conv(ch[1], ch[2], 3)
    self.path1_3_1 = conv(ch[2], ch[3], (1,3))
    self.path1_3_2 = conv(ch[2], ch[4], (3,1))
    self.path2_1 = conv(ch[5], ch[6], 1)
    self.path2_2_1 = conv(ch[6], ch[7], (1,3))
    self.path2_2_2 = conv(ch[6], ch[8], (3,1))
    self.path3_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    self.path3_2 = conv(ch[9],ch[10],1)
    self.path4 = conv(ch[11],ch[12],1)
  def forward(self,x):
    x1_1 = self.path1_3_1(self.path1_2(self.path1_1(x)))
    x1_2 = self.path1_3_2(self.path1_2(self.path1_1(x)))
    x2_1 = self.path2_2_1(self.path2_1(x))
    x2_2 = self.path2_2_2(self.path2_1(x))
    x3 = self.path3_2(self.path3_1(x))
    x4 = self.path4(x)
    return torch.cat([x1_1,x1_2,x2_1,x2_2,x3,x4],1)

class InveptionV2(nn.Module):
  def __init__(self,num_classes,init_weights=True):
    super(InceptionV2,self).__init__()
    self.conv1 = nn.Conv2d(3,32,3,stride=2,padding=0)
    self.conv2 = nn.Conv2d(32,32,3,stride=1,padding=0)
    self.conv3 = nn.Conv2d(32,64,3,stride=1,padding=2)
    self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=3)
    self.conv4 = nn.Conv2d(64,80,3,stride=1,padding=0)
    self.conv5 = nn.Conv2d(80,192,3,stride=2,padding=0)
    self.conv6 = nn.Conv2d(192,288,3,stride=1,padding=1)
    ch1=[288]
    ch2=[768]
    ch3=[1280]
    self.inception1=Incep1()
    self.inception2=Incep2()
    self.inception3=Incep3()
    self.pool2 = nn.AvgPool2d(kernel_size=8)
    self.dropout = nn.Dropout(p=0.4)
    self.fc1 = nn.Linear(2048,num_classes)
    
    if init_weights:
      self._initialize_weights()
    
  def forward(self,x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.pool1(x)
    x = self.conv4(x)
    x = self.conv5(x)
    x = self.conv6(x)
    for i in range(3):
      x = self.inception1(x,ch1)
    for i in range(5):
      x = self.inception2(x,ch2)
    for i in range(2):
      x = self.inception3(x,ch3)
    x = self.pool2(x)
    x = x.view(x.shape[0],-1)
    x = self.dropout(x)
    x = self.fc1(x)
    return x

  def _initialize_weights(self):
    for m in self.modules():
      if isinstance(m,nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out',nonlinearity='relu')
        if m.bias is not None:
          nn.init.constant_(m.bias,0)
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias,0)
      elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0 ,0.01)
        nn.init.constant_(m.bias,0) 
  